In [1]:
import tensorflow as tf
from functools import reduce
from numpy import unique, array, vectorize
from sklearn.metrics import accuracy_score, f1_score

class SVMClassifier:

    def __init__(self, train_data=None):
        data, labels = train_data

        labels = self._transform_labels(labels)
        data = self._flatten_input(data)
        
        self.train_data = (data, labels)

        self.assemble_graph()

        self._open_session()

        if train_data:
            self.train()     

    def assemble_graph(self, learning_rate = 0.02):
        data, labels = self.train_data
        in_size = [None, data.shape[1]]
        out_size = [None, labels.shape[1]]
        
        self.input_data = tf.placeholder(tf.float32, shape=in_size, name="input_data")
        self.output_labels = tf.placeholder(tf.float32, shape=out_size, name="output_labels")
        
        weight = tf.get_variable("weight", (in_size[1], 1), tf.float32)
        bias = tf.get_variable("bias", (1,1), tf.float32)
        
        inference = tf.matmul(self.input_data,weight) - bias
        
        self.output = tf.sign(inference)
        
        self.loss = tf.reduce_mean(tf.maximum(0., 1. - self.output_labels * inference))
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        self.training = optimizer.minimize(self.loss)
        

    def train(self, epochs=20, minibatch_size=256):
        minibatches = self._create_minibatches(minibatch_size)
        sess = self.sess
        sess.run(tf.global_variables_initializer())
        
        for e in range(epochs):
            for data, labels in minibatches:
                sess.run(self.training, {self.input_data: data, self.output_labels: labels})

            loss_val = sess.run(self.loss, {self.input_data: minibatches[0][0], self.output_labels: minibatches[0][1]})
            print("Epoch %d, loss: %.2f" % (e, loss_val))

    def predict(self, data):
        data = self._flatten_input(data)
        
        predict = self.sess.run(self.output, {self.input_data: data})
        
        for i in range(len(predict)):
            if predict[i] == -1:
                predict[i] = 0
        return predict

    def _create_minibatches(self, minibatch_size):
        pos = 0

        data, labels = self.train_data
        n_samples = len(labels)

        batches = []
        while pos + minibatch_size < n_samples:
            batches.append((data[pos:pos+minibatch_size,:], labels[pos:pos+minibatch_size]))
            pos += minibatch_size

        if pos < n_samples:
            batches.append((data[pos:n_samples,:], labels[pos:n_samples]))

        return batches

    def _transform_labels(self, labels):
        labels = labels.reshape((-1, 1))
        for i in range(len(labels)):
            if labels[i] == 0:
                labels[i] = -1
        return labels
        

    def _flatten_input(self, data):
        size = data.shape[1] * data.shape[2]
        data = data.reshape(data.shape[0], size)
        return data

    def _open_session(self):
        self.sess = tf.Session()





if __name__ == "__main__":



    def mnist_to_binary(train_data, train_label, test_data, test_label):

        binarized_labels = []
        for labels in [train_label, test_label]:
            remainder_2 = vectorize(lambda x: x%2)
            binarized_labels.append(remainder_2(labels))

        train_label, test_label = binarized_labels

        return train_data, train_label, test_data, test_label




    ((train_data, train_labels),
        (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

    train_data, train_labels, test_data, test_labels = mnist_to_binary(train_data, train_labels, eval_data, eval_labels)

    svm = SVMClassifier((train_data, train_labels))
    print("Testing score f1: {}".format(f1_score(test_labels, svm.predict(test_data))))



Epoch 0, loss: 188.64
Epoch 1, loss: 185.67
Epoch 2, loss: 206.55
Epoch 3, loss: 213.81
Epoch 4, loss: 213.91
Epoch 5, loss: 209.60
Epoch 6, loss: 283.82
Epoch 7, loss: 251.64
Epoch 8, loss: 241.48
Epoch 9, loss: 204.78
Epoch 10, loss: 329.82
Epoch 11, loss: 189.57
Epoch 12, loss: 214.51
Epoch 13, loss: 226.71
Epoch 14, loss: 224.63
Epoch 15, loss: 214.44
Epoch 16, loss: 208.76
Epoch 17, loss: 212.81
Epoch 18, loss: 265.86
Epoch 19, loss: 207.91
Testing score f1: 0.8922607118999905
